In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import torch

from dataloader import MDITDataset
from torch.utils.tensorboard import SummaryWriter
from model import MDIT

from metrics import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# device = 'cpu'
print(device)

cuda:0


In [3]:
trainset = MDITDataset(data_dir='./processed_data/train/')
testset = MDITDataset(data_dir='./processed_data/test/')

Found 12800 files in ./processed_data/train/.
Found 3200 files in ./processed_data/test/.


In [4]:
print(len(testset), len(trainset))

3200 12800


In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=20, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(testset,batch_size=1)

In [6]:
trainsummary = SummaryWriter(log_dir='./logs/train/')
testsummary = SummaryWriter(log_dir='./logs/test/')


In [7]:
EPOCHS = 100
model = MDIT().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
# criterion = torch.nn.CrossEntropyLoss()
pbar = tqdm(total=len(trainloader) * EPOCHS)

train_loss = []
model.train()
model.to(device)
for epoch in range(EPOCHS):
    for i, data in enumerate(trainloader):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.to(device), labels.to(device)
        pbar.update(1)
        outputs = model(inputs)

        optimizer.zero_grad()
        # print(outputs.shape, labels.shape)
        # outputs = outputs.float()
        # labels = labels.float()
        # loss = criterion(outputs.float(), labels.float())
        loss = adaptive_maxpool_loss(labels, outputs)

        # loss = adaptive_maxpool_loss(labels, outputs)
        loss.backward()
        optimizer.step()

        if(i%100 == 0):
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }
            torch.save(checkpoint, 'checkpoint.pth')
            print("Checkpoint saved")


        if(i % 100 == 0):
            
            print("Epoch: ", epoch, " Iteration: ", i)
            print(loss)
            F1 = f1(outputs, labels)
            print(F1)
            precs = precision(outputs, labels)
            print(precs)
            rec = recall(outputs, labels)
            print(rec)
            trainsummary.add_scalar('Loss', loss.item(), epoch * len(trainloader) + i)
            trainsummary.add_scalar('F1', F1, epoch * len(trainloader) + i)
            trainsummary.add_scalar('Precision', precs, epoch * len(trainloader) + i)
            trainsummary.add_scalar('Recall', rec, epoch * len(trainloader) + i)
            
            model = model.eval()
            val_loss = []
            val_prec = []
            val_rec = []
            val_f1 = []
            with torch.no_grad():
                for j, data in enumerate(testloader):
                    
                    if (j > 200):
                        break

                    inputs, labels = data
                    if torch.cuda.is_available():
                        inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    # loss = criterion(outputs.float(), labels.float())
                    loss = adaptive_maxpool_loss(labels, outputs)
                    F1 = f1( labels, outputs)
                    precs = precision(labels, outputs)
                    rec = recall(labels, outputs)
                    val_loss.append(loss.item())
                    val_prec.append(precs.item())
                    val_rec.append(rec.item())
                    val_f1.append(F1.item())
            model = model.train()
            # print("Validation Loss: ", np.mean(val_loss))
            # print("Validation Precision: ", np.mean(val_prec))
            # print("Validation Recall: ", np.mean(val_rec))
            # print("Validation F1: ", np.mean(val_f1))
            testsummary.add_scalar('Loss', np.mean(val_loss), epoch * len(trainloader) + i)
            testsummary.add_scalar('F1', np.mean(val_f1), epoch * len(trainloader) + i)
            testsummary.add_scalar('Precision', np.mean(val_prec), epoch * len(trainloader) + i)
            testsummary.add_scalar('Recall', np.mean(val_rec), epoch * len(trainloader) + i)

        

        

  0%|          | 0/64000 [00:00<?, ?it/s]

Checkpoint saved
Epoch:  0  Iteration:  0
tensor(0.9199, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.0181, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(0.2359, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.0094, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Checkpoint saved
Epoch:  0  Iteration:  100
tensor(0.0077, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0., device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0., device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Checkpoint saved
Epoch:  0  Iteration:  200
tensor(0.0091, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0., device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0., device='cuda:0', dtype=torch.